In [12]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import pickle
import requests
import re
import numpy as np

alphabet = 'abcdefghijklmnopqrstuvwxyz'.upper()
games = ['WC1','WC2','WC2X','WC3','WC3X','WoW','WoWBC','WoWWotLK','WoWC','WoWMoP','WoWWoD','WoWL','WoWBfA','WoWS','WoWD']

In [13]:
def get_metadata():
    meta_data = []
    
    # get meta data of all major characters

    url = "https://wowpedia.fandom.com/wiki/Major_characters"
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    soup_a = soup.select("p > small")
    soup_b = soup.select("p > a")
    for i,s in enumerate(soup_a):
        d = {}

        expansions = s.text.replace(",","").replace("—","").split()
        expansions = [int(1) if i in expansions else int(0) for i in games]

        text = soup_b[i]['href'][6:]
        name = text.replace("%27", "'")

        if 1 in expansions:
            d['Name'] = name
            for j,g in enumerate(games):
                d[g] = int(expansions[j])

    
            meta_data.append(d)
    
    return meta_data

In [14]:
meta_data = get_metadata()
df_meta = pd.DataFrame(meta_data, dtype=object)
# saving the df's
with open('df_meta.pickle', 'wb') as handle:
    pickle.dump(df_meta, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_meta

,Name,WC1,WC2,WC2X,WC3,WC3X,WoW,WoWBC,WoWWotLK,WoWC,WoWMoP,WoWWoD,WoWL,WoWBfA,WoWS,WoWD
0,Terokk,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,Talon_King_Ikiss,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,Archimonde,0,0,0,1,0,0,1,1,0,0,1,1,0,0,0
3,Kil'jaeden,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0
4,Mannoroth,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,Zul,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
268,Talanji,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
269,Vilnak'dor,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
270,Helya,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [15]:
def get_characters():
    characters = []
    
    # get meta data of all major characters

    url = "https://wowpedia.fandom.com/wiki/Major_characters"
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    soup_a = soup.select("p > small")
    soup_b = soup.select("p > a")
    for i,s in enumerate(soup_a):
        d = {}

        expansions = s.text.replace(",","").replace("—","").split()
        expansions = [int(1) if i in expansions else int(0) for i in games]

        text = soup_b[i]['href'][6:]
        name = text.replace("%27", "'")

        if 1 in expansions:
            d['Name'] = name
            
            characters.append(d)
#     # get lore characters

#     c_list = []

#     for letter in tqdm(alphabet):
#         url = "https://wowpedia.fandom.com/wiki/Category:Lore_characters?pagefrom={}".format(letter)
#         html = urlopen(url).read()
#         soup = BeautifulSoup(html, features="html.parser")

#         for i in soup.find_all('li'):
#             text = i.find('a')['href']
#             verboten = ['/wiki/Genealogy', '/wiki/Local_Sitemap']
#             if text[0:5] == '/wiki' and ":" not in text and text not in verboten:
#                 c_list.append(text.replace("%27", "'").replace("%22", '"').split("/wiki/")[1])

#     c_list = sorted(list(set(c_list)))

#     for name in c_list:
#         d = {}
#         d['Name'] = name

#         characters.append(d)
    
    return characters

In [16]:
characters = get_characters()
df = pd.DataFrame(characters)
# saving the df's
with open('df.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)
df


,Name
0,Terokk
1,Talon_King_Ikiss
2,Archimonde
3,Kil'jaeden
4,Mannoroth
...,...
267,Zul
268,Talanji
269,Vilnak'dor
270,Helya


In [17]:
def get_wikicode(t):

    response = requests.get(
         'https://wowpedia.fandom.com/api.php',
             params={
             'action': 'query',
             'format': 'json',
             'titles': '{}'.format(t),
             'prop': 'revisions',
             'rvprop': 'content',
         }
     ).json()


    page = next(iter(response['query']['pages'].values()))

    wikicode = page['revisions'][0]['*']
    
    return wikicode

In [33]:
import pickle

with open('chars2.pickle', 'rb') as handle:
    characters = pickle.load(handle)
print(len(characters))

2333


In [19]:
# characters = []
with open('chars2.pickle', 'rb') as handle:
    characters = pickle.load(handle)

# for i,name in enumerate(tqdm(list(df['Name']))):
for name in tqdm(list(df['Name'].iloc[255:])):
    
#     if i % 20 == 0:
#         characters = list(np.unique(characters))
        
#         with open('characters.pickle', 'wb') as handle:
#             pickle.dump(characters, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
    text = get_wikicode(name)

    WORD_RE = re.compile(r"\[{2}.*?\]{2}")
    for word in list(np.unique(WORD_RE.findall(text))):
        try:
            char = word.replace(" ", "_")[2:-2]
            if "Category" not in char and ":" not in char and "#" not in char and "|" not in char:
                url = "https://wowpedia.fandom.com/wiki/{}".format(char)
                html = urlopen(url).read()
                soup = BeautifulSoup(html, features="html.parser")

                table =  soup.find("table",{"class":"infobox darktable"})

                if len(table) > 0:

                    text = table.text
                    if "Gender" not in text:
                        pass
                    else:
                        characters.append(char)

        except:
            pass

characters = list(np.unique(characters))

# df = df.append(pd.DataFrame(characters, columns=['Name']), ignore_index=True)

In [28]:
filter_characters = []

for char in characters:
    if "#" not in char and "|" not in char and "_(tactics)" not in char:
        filter_characters.append(char)

In [35]:
len(filter_characters)

2253

In [36]:
df = df.append(pd.DataFrame(filter_characters, columns=['Name']), ignore_index=True)

In [32]:
list(asdasdas['Name'])

['Terokk',
 'Talon_King_Ikiss',
 'Archimonde',
 "Kil'jaeden",
 'Mannoroth',
 'Balnazzar',
 'Magtheridon',
 "Mal'Ganis",
 'Tichondrius',
 'Varimathras',
 'Azgalor',
 'Hakkar_the_Houndmaster',
 'Kazzak',
 'Velen',
 'Akama',
 'Nobundo',
 'Vindicator_Maraad',
 'Yrel',
 'Alexstrasza',
 'Kalecgos',
 'Malygos',
 'Deathwing',
 'Daval_Prestor',
 'Nozdormu',
 'Ysera',
 'Korialstrasz',
 'Eranikus',
 'Sintharia',
 'Tyranastrasz',
 'Nefarian',
 'Onyxia',
 'Katrana_Prestor',
 'Wrathion',
 'Sabellian',
 'Arygos',
 'Kirygosa',
 'Chronormu',
 'Galakrond',
 'Kairozdormu',
 'Tyrygosa',
 'Sapphiron',
 'Ebyssian',
 'Dargrul',
 'Navarrogg',
 'Brann_Bronzebeard',
 'Muradin_Bronzebeard',
 'Magni_Bronzebeard',
 'Moira_Thaurissan',
 'Dagran_Thaurissan_II',
 'Thargas_Anvilmar',
 'Falstad_Wildhammer',
 'Kurdran_Wildhammer',
 'Dagran_Thaurissan',
 'Sorcerer-thane_Thaurissan',
 "Kael'thas_Sunstrider",
 'Aethas_Sunreaver',
 'Grand_Magister_Rommath',
 'Halduron_Brightwing',
 'Lady_Liadrin',
 "Lor'themar_Theron",
 'Va

In [ ]:
# saving the df's
with open('df.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)